In [1]:
import string  as strs
from sklearn.naive_bayes import GaussianNB as gnb
import numpy as np
import chardet as chd
!pip install nltk
import nltk as nlp

You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [415]:
stop  = nlp.corpus.stopwords.words(('english'))
def cleaner(filename):
    words = []
    try:
        with open(filename,'r') as f:
            for x in f.readlines():
                word = nlp.tokenize.word_tokenize(x)
                word = [w for w in word if w  not in strs.punctuation]
                try:
                    if(word[0] == "Subject"):
                        del word[0]
                except IndexError:
                    word = word
                word = [w for w in word if w not in stop]
                words.append(word)
    except UnicodeDecodeError:
        return []
    return words

In [102]:
import glob
x = []
for i in glob.glob('spam/*.txt'):
    x.append(cleaner(i))

In [103]:
spamtokens = []
for i in x:
    for y in i:
        for z in y:
            spamtokens.append(z)

In [104]:
spamtokens = np.array(spamtokens)

In [105]:
unqspamtokens = np.array(list(set(spamtokens)))

In [106]:
spamtokens.size

196588

In [107]:
distspam = nlp.FreqDist(spamtokens)

In [108]:
distunqspam = nlp.FreqDist(unqspamtokens)

In [109]:
distspam

FreqDist({'3': 1239, 'com': 990, 'http': 981, '2': 858, 'company': 728, '1': 719, '0': 698, 'e': 638, '00': 585, 'www': 585, ...})

In [110]:
updistspam = nlp.probability.LaplaceProbDist(distspam)

In [111]:
tokens = []
for i in glob.glob('ham/*.txt'):
    tokens.append(cleaner(i))

In [112]:
x = []
for i in tokens:
    for w in i:
        for y in w:
            x.append(y)
hamtokens = np.array(x)

In [120]:
hamtokens.size
distham = nlp.FreqDist(hamtokens)

In [114]:
etokens = np.concatenate((spamtokens,hamtokens))

In [115]:
edist = nlp.FreqDist(etokens)

In [116]:
distham = nlp.FreqDist(hamtokens)
distham

FreqDist({'ect': 13897, 'hou': 7281, 'enron': 6555, '2000': 4308, '``': 2872, 'gas': 2861, 'deal': 2789, 'subject': 2731, 'com': 2717, 'please': 2715, ...})

In [117]:
upedist = nlp.probability.LaplaceProbDist(edist)

In [121]:
unqhamtokens = np.array(list(set(hamtokens)))

In [150]:
unqdistham = nlp.FreqDist(unqhamtokens)

In [151]:
probspam = []
for i,j in distunqspam.items():
    probspam.append(j/edist[i])

In [152]:
targetspam = []
for x in probspam:
    targetspam.append('spam')

In [153]:
probham = []
for i,j in unqdistham.items():
    probham.append(j/edist[i])

In [154]:
targetham = []
for x in probham:
    targetham.append('ham')

In [254]:
wordtable = []
for x,y,z in zip(unqspamtokens,probspam,targetspam):
    wordtable.append([x,y,z])

In [256]:
wordtableham = []
for x,y,z in zip(unqhamtokens,probham,targetham):
    wordtableham.append([x,y,z])

In [281]:
import pandas as pd

In [282]:
data =pd.DataFrame(data =wordtable,columns=['WordSpam','ProbabilitySpam','Label1'])

In [283]:
data.head()

,WordSpam,ProbabilitySpam,Label1
0,cli,0.166667,spam
1,airstrip,1.000000,spam
2,filename,0.142857,spam
3,miilennium,0.142857,spam
4,breakthrough,0.333333,spam


In [284]:
dataham = pd.DataFrame(data=wordtableham,columns=['Wordham','ProbabilityHam','Label2'])

In [285]:
final = data.append(dataham) 

In [286]:
final[['ProbabilityHam']] = final[['ProbabilityHam']].fillna(0)

In [287]:
len(set.intersection(set(unqspamtokens),set(unqhamtokens)))

8298

In [288]:
final[['ProbabilitySpam']] = final[['ProbabilitySpam']].fillna(0)

In [289]:
final['Wordham'] = final['Wordham'].fillna(final['WordSpam'])

In [290]:
final['WordSpam'] = final['WordSpam'].fillna(final['Wordham'])
final['Label1'] = final['Label1'].fillna(final['Label2'])
final['Label2'] = final['Label2'].fillna(final['Label1'])

In [291]:
del final['Wordham']

In [295]:
del final['Label1']

In [296]:
final.columns = ['Label','ProbabilityHam','ProbabilitySpam','Word']

In [297]:
final.head()

,Label,ProbabilityHam,ProbabilitySpam,Word
0,spam,0.0,0.166667,cli
1,spam,0.0,1.000000,airstrip
2,spam,0.0,0.142857,filename
3,spam,0.0,0.142857,miilennium
4,spam,0.0,0.333333,breakthrough


In [298]:
x = final['ProbabilityHam'].values
np.append(x,final['ProbabilitySpam'].values)

array([0., 0., 0., ..., 0., 0., 0.])

In [299]:
y = final['Label']

In [308]:
final = final.sample(frac=1).reset_index(drop=True)
final.head()

,Label,ProbabilityHam,ProbabilitySpam,Word
0,spam,0.0,0.019608,executive
1,ham,0.5,0.000000,991107
2,spam,0.0,0.333333,bourn
3,spam,0.0,0.500000,beloved
4,spam,0.0,1.000000,rht


In [321]:
x = final.iloc[:,1:].values
y = final['Label'].values

In [322]:
from sklearn.model_selection import train_test_split as tts

In [323]:
x_train,x_test,y_train,y_test = tts(x,y)

In [394]:
regressor = gnb()
regressor.fit(x_train[:,:2],y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [398]:
regressor.predict(x_test[1:10,:2])

array(['spam', 'ham', 'ham', 'spam', 'spam', 'ham', 'ham', 'spam', 'spam'],
      dtype='<U4')

In [340]:
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB

In [341]:
regressor2 = SGDClassifier(loss='log')

In [389]:
regressor2.fit(x_train[:,:2],y_train)

SGDClassifier(alpha=0.0001, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='log', max_iter=None,
       n_iter=None, n_iter_no_change=5, n_jobs=None, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False)

In [411]:
feature = np.array([[0.09082340238405234,0.04232359862934412]])
print(regressor2.predict(x_test[1:10,:2]))
print(regressor.predict(x_test[1:10,:2]))
print(regressor.predict(feature))
print(regressor2.predict(feature))

['spam' 'ham' 'ham' 'spam' 'spam' 'ham' 'ham' 'spam' 'spam']
['spam' 'ham' 'ham' 'spam' 'spam' 'ham' 'ham' 'spam' 'spam']
['ham']
['ham']


In [401]:
classi = MultinomialNB()
classi.fit(x_train[:,:2],y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [412]:
print(classi.predict(x_test[1:10,:2]))
print(classi.predict(feature))

['spam' 'ham' 'spam' 'spam' 'spam' 'ham' 'ham' 'spam' 'spam']
['spam']


In [405]:
print(y_test[1:10])

['spam' 'ham' 'ham' 'spam' 'spam' 'ham' 'ham' 'spam' 'spam']


array([[0.0, 1.0, 'bbcja'],
       [0.0, 0.5, 'sy'],
       [0.5, 0.0, '2666'],
       ...,
       [0.0, 0.25, 'abused'],
       [0.0, 1.0, 'ros'],
       [0.3333333333333333, 0.0, 'workplace']], dtype=object)

In [421]:
tokens = []
for x in cleaner('Check.txt'):
    for y in x:
        tokens.append(y)
check = np.array(tokens)

In [424]:
checktable = nlp.FreqDist(check)

0.03389830508474576

In [480]:
features = []
for x in checktable.keys():
    if x in hamtokens and x in spamtokens:
        features.append([checktable[x]/distham[x],checktable[x]/distspam[x],x])
    else:
        if x in hamtokens:
            features.append([checktable[x]/distham[x],0,x])
        elif x in spamtokens:
            features.append([0,checktable[x]/distspam[x],x])
        else:
            features.append([tokens.count(x)/len(tokens),tokens.count(x)/len(tokens),x])
            print(x)
print(len(features))
features = pd.DataFrame(features)
features.head()

BI
State
Bank
India
SBI
The
alerted
Twitter
Received
Income
Tax
Department
These
fraudsters
Ensure
''
tweet
Taxpayers
Beware
Please
phishing
CBIC
Infosys_GSTN
Visit
gst.gov.in
GST
All
This
Read
More
Unsubscribe
74


,0,1,2
0,0.008475,0.008475,BI
1,0.000000,1.000000,warns
2,0.030534,0.042553,customers
3,3.000000,3.000000,fake
4,0.142857,0.333333,income


In [481]:
prediction1 = regressor.predict(features.values[:,:2])
prediction2 = regressor2.predict(features.values[:,:2])

In [482]:
prediction3 = classi.predict(features.values[:,:2])

In [501]:
spammer = 0
hamming = 0
for x in prediction1:
    if (x in prediction3 or x in prediction2) and x == 'spam':
        spammer+=1
    elif (x in prediction3 or x in prediction2) and x == 'ham':
        hamming+=1
print(spammer,hamming)

31 43


In [506]:
if hamming > spammer:
    print("Mail is ham with {}%".format(hamming/(spammer+hamming)*100))

Mail is ham with 58.108108108108105%
